The code beneath generates a 6 digit OTP and uploads it to firestore for multiputpose use e.g Transaction verification OTP, access code generation, 2-factor auth, etc. It can be deployed as a cloud function on Azure, GCP, or AWS, or a self-managed service.
There are three functions defined:

1)generate_code: serves as the 'main' function orchestrates the process of generating the code and uploading it to the DB

2)generate_random_no: This function generates a random n-digit number by joining together random n-digits into a single number.

3)post_to_db: THis function uploads the generated n-digit number to the firestore database.

The code can be adjusted to accomodate user info i.e. the n-digit code is posted to the DB along with information such as username, email, location, etc

In [ ]:
!pip install exponent_server_sdk
!pip install --upgrade firebase-admin

In [ ]:
#CODEGEN
  
def generate_code(context):
  '''generates and posts a unique 6 digit code (OTP) to a database  
  '''
  code_str= generate_random_no(n_digits= 6)
  print(code_str)
  print("posted to db", post_to_db(code_str))
  return str(code_str)
  
def generate_random_no(n_digits= 6):
  '''uses random numbers to generate a unique n-digit code'''
  import numpy as np

  numbers = []
  for i in range(n_digits):
    numbers.append(int(np.floor(np.random.rand()*10)))
  code = 0
  code_str = ''
  for i in range(0,n_digits,1):
    code += numbers[i]*10**(n_digits-i-1)
    code_str = code_str + str(numbers[i])
  print(code_str)
  return code_str

def post_to_db( code_str):
  '''post code along with user data in code db'''
  #initialise firebase
  import firebase_admin
  from firebase_admin import credentials
  from firebase_admin import firestore

  # Use a service account
  cred = credentials.Certificate('/shopmob/shopmob_service_acc')
  try:
    firebase_admin.initialize_app(cred)
  except:
    print('error occurred in initialising firebase. Perhaps already initialised')

  try:
  db = firestore.client()
  #post to db --- uncomment fields to include user details
  data = {
  #u'uname': data['userId'],
  #u'ID': data['userName'],
  u'code': code_str 
        }
  response = db.collection(u'codegen_codes').document(code_str).set(data)
  except:
    print("error occured in uploding the generated code")
  return response





In [ ]:
generate_code("<userDetails>")